In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
from node import Node
import pandas as pd
import numpy.linalg as la

In [2]:
#Circles test case

# observed = np.array([1.0, 1.0, 1.0, 1.0, 1.0,
#                      1.0, 2.0, 2.0, 2.0, 1.0, 
#                      1.0, 2.0, 2.0, 2.0, 1.0, 
#                      1.0, 2.0, 2.0, 2.0, 1.0, 
#                      1.0, 1.0, 1.0, 1.0, 2.0])

# expected = np.ones(25)

In [3]:
#Original test case (3 random elements):

# expected = np.random.normal(100, 20, 3)
poissons = np.vectorize(lambda x: np.random.poisson(x))
# observed = poissons(expected)


# expected = np.array([86.5, 113.55, 101.32])
# observed = np.array([265, 345, 324])
# observed = np.array([50, 345, 324])

In [4]:
#5-element test case:

# expected = np.random.normal(100, 50, 6)
# observed = poissons(expected)
# observed = np.array([100, 170,  100,  100,  200])
# expected = np.ones(5)*100

In [5]:
path = r"C:\Users\joles\OneDrive\Documents\ML4G\train.csv"
path2 = r"C:\Users\joles\OneDrive\Documents\ML4G\mnist_bin.csv"

theta = pd.read_csv(path)
labels = theta['label']
# theta = theta.drop(['label'], axis=1)
theta = pd.read_csv(path2)

theta.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# theta = theta.drop(['Unnamed: 0'], axis=1)
theta.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
subset = theta.iloc[-1]
lbl = subset[0]
subset = subset[1:]

In [8]:
theta = theta.drop([41999],axis=0)

In [9]:
#Build Tree from MNIST

tree = Node(theta= theta, name= '*')
tree.build_tree(theta=theta, min_filters_to_split=10000)

In [10]:
# tree.left.left.left.theta
# tree.left.left.right.theta
# tree.left.right.left.theta
# tree.left.right.right.theta
# tree.right.left.theta
# tree.right.right.theta

#TREE SHAPE: 

#                          ROOT
#                      /        \
#                      L         R     L1
#                   /    \      / \
#                  L      R    L   R   L2
#                 / \    / \
#                L   R  L   R          L3

In [11]:
def llri(obs, exp, q): #score function for poisson (one element)
    return obs*np.log(q) + exp*(1-q)

def llri_pen(obs, exp, q, delta): #penalized score function for poisson (one element)
    return obs*np.log(q) + exp*(1-q) - delta

llr = np.vectorize(llri) #generate unpenalized scores for a given q value

llr_pen = np.vectorize(llri_pen) #genereate penalized scores for a given q value

In [12]:
#DEFINED FUNCTIONS FOR USE IN ALGORITHM:

#BINARY SEARCH FUNCTIONS: ################################################
def q_min(obs, exp):

    minimum = 0.000001
    qmle = 1

    while abs(qmle - minimum) > 0.00000001:
        q_mid = (minimum + qmle)/2

        if llri(obs, exp, q_mid) > 0:
            qmle = qmle - (qmle - minimum)/2
        else:
            minimum = minimum + (qmle - minimum)/2
    return (minimum + qmle)/2

def q_max(obs, exp):

    maximum = 10000000
    qmle = 1

    while abs(maximum - qmle) > 0.000001:
        q_mid = (maximum + qmle)/2

        if llri(obs, exp, q_mid) < 0:
            maximum = maximum - (maximum-qmle)/2
        else:
            qmle = qmle + (maximum-qmle)/2

    return (maximum + qmle)/2

###################################################
def q_min_pen(obs, exp, delta):  #need to fix


    minimum = 0.000001
    qmle = 1

    while abs(qmle - minimum) > 0.00000001:
        q_mid = (minimum + qmle)/2

        if llri_pen(obs, exp, q_mid, delta) > 0:
            qmle = qmle - (qmle - minimum)/2
        else:
            minimum = minimum + (qmle - minimum)/2
    return (minimum + qmle)/2
####################################################

def q_max_pen(obs, exp, delta):

    maximum = 10000000
    qmle = 1

    while abs(maximum - qmle) > 0.000001:
        q_mid = (maximum + qmle)/2

        if llri_pen(obs, exp, q_mid, delta) < 0:
            maximum = maximum - (maximum-qmle)/2
        else:
            qmle = qmle + (maximum-qmle)/2

    return (maximum + qmle)/2
######################################################################################


#FOR FINDING Q-INTERVALS: ###################################################################################
def minmax(obs, exp):
    return (q_min(obs, exp), q_max(obs, exp))

def minmax_pen(obs, exp, delta):
    return (q_min_pen(obs, exp, delta), q_max_pen(obs, exp, delta))

#Get q intervals
def get_INTERVALS(obs, exp, delta):
    return np.array([q_min(obs, exp), q_min_pen(obs, exp, delta), q_max_pen(obs, exp, delta), q_max(obs, exp)])

get_all_intervals = np.vectorize(get_INTERVALS)
#############################################################################################################

def relu_scores(scores, delta):
    return min(np.abs(scores), delta)

def ReLU(scores):
    return (scores > 0).view('i1')


def compute_new_subset(scores, filter, delta):
    
    subset = np.zeros(len(filter))
    weights = np.zeros(len(filter))
    penalties = np.zeros(len(filter))

    for i in range(len(filter)):

        if (scores[i] > delta):

            if filter[i] == 0:
                penalties[i] = delta
            
            subset[i] = 1 #always include element in subset
            weights[i] = delta #w_i = delta

        elif (np.abs(scores[i]) <= delta):

            subset[i] = filter[i] #include iff in filter
            weights[i] = np.abs(scores[i]) #w_i = score_i

        else:

            if filter[i] == 1: #penalize by delta if in filter
                penalties[i] = delta
                
            subset[i] = 0 #always exclude from subset
            weights[i] = delta #w_i = delta
            
    return subset, weights

def GRAD_F(obs, exp, q):
    return (obs/q) - exp

Grad_vec = np.vectorize(GRAD_F)

def compute_qmle(obs, exp, guess):
    q = guess
    numiter = 0
    prev = 0
    while la.norm(np.abs(llr(obs, exp, q) - prev)) > 1e-10:
    # while la.norm(Grad_vec(obs, exp, q)) > 1e-5:
        prev = llr(obs, exp, q)
        q = q + 0.001*(sum(Grad_vec(obs, exp, q)))
        numiter = numiter + 1
        
        # print("Current q: {}, Function Value: {}, F(q+1) - F(q): {}".format(q, llr(obs, exp, q), np.abs(llr(obs, exp, q) - prev)))
    return q


In [16]:
observed = subset*2
expected = subset
delta = 1

In [17]:
for i in range(len(observed)):
    if i == 0:
        intervals = get_INTERVALS(observed[i], expected[i], delta)
    else:
        intervals = np.append(intervals, get_INTERVALS(observed[i], expected[i], delta))     

intervals = np.unique(intervals)

# filter1 = np.array([0, 1, 1, 0, 1])
# filter2 = np.array([1, 0, 1, 0, 0])
# filter3 = np.array([0, 1, 0, 1, 0])
# filter4 = np.array([1, 1, 1, 0, 1])
# filter5 = np.array([1, 0, 0, 0, 1])

# theta = pd.DataFrame([filter3, filter2, filter1, filter4, filter5])
# tree = Node(theta= theta, name= '*')
# tree.build_tree(theta=theta, min_filters_to_split=100)

In [18]:
intervals

array([9.99999996e-01, 1.00000028e+00, 3.51286227e+00, 1.00000000e+07])

In [19]:
#ALGORITHM:
###########################################################################

delta = 1
F_MAX = 0

for i in range(len(intervals) - 1):

    converged = False

    qmid = (intervals[i] + intervals[i+1])/2

    print("Initial q value: {}".format(qmid))

    while converged == False:

        scores = llr(observed, expected, qmid)
        # print("Scores for data elements: {}".format(scores))

        S = ReLU(scores=scores) #initial subset
        # print("initial subset: {}".format(S))

        # weights = np.ones(len(initial)) #chg
        weights = [delta if np.abs(score) > delta else np.abs(score) for score in scores]
        # print("Weights for initial subset: {}".format(weights))

        Fmax = sum(scores[scores > 0])
        print("Score of initial subset: {}".format(Fmax))

        filter, distance = tree.traverse(S, weights=weights)
        # print("Best matched filter for subset: {}".format(filter))
        
        pens = np.minimum(np.abs(scores), delta)
        
        F = Fmax - sum(pens[filter != S]) #new score
        print("New score: {}".format(F))

        if F > F_MAX:
            F_MAX = F
            BEST_S = S
            BEST_F = filter

        S, W = compute_new_subset(scores=scores, filter=filter, delta=delta)
        print("New subset: {}".format(S))
        print("Weights for new S: {}".format(W))

        newS = scores
        newEx = expected.copy()
        newObs = observed.copy()
        newS[S == 0] = 0
        newEx[S == 0] = 0
        newObs[S == 0] = 0
        
        qmle =  sum(newObs)/(sum(newEx) + 10e-6) #compute_qmle(newS, expected, 1) #compute qmle of new S
        print("Q_MLE for new subset: {}".format(qmle))
        print("#####################################################################")

        
        
        if np.abs(qmle - qmid) < 0.001:
            converged = True
        else:
            qmid = qmle
                
print("Subset: {} \nBest Matched Filter:\n{} \nScore: {}".format(BEST_S, BEST_F, F_MAX))

Initial q value: 1.0000001402458896
Score of initial subset: 1.7390487867523528e-05


IndexError: list index out of range

In [281]:
#Notes for the above cell: we create a new subset from scratch every iteration of the while loop based on the latest value of q
#if q or the subset doesnt change, then we break out of the loop and test the next interval